**Multi-knapsack Problem with Bandwidth Constraint**

In [ ]:
# Import all required liberaries
import random
from random import randint
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
# Create the graph

G = nx.DiGraph()

# total demand: 1,2,3,4,5 = 15000

G.add_edge("s", "a", weight=1000)
G.add_edge("s", "b", weight=6000)
G.add_edge("s", "c", weight=8000)
G.add_edge("a", "d", weight=3000)
G.add_edge("a", "e", weight=1000)
G.add_edge("b", "e", weight=8000)
G.add_edge("b", "f", weight=6000)
G.add_edge("c", "f", weight=1500)
G.add_edge("c", "d", weight=8000)
G.add_edge("d", "t", weight=8000)
G.add_edge("e", "t", weight=1000)
G.add_edge("f", "t", weight=6000)

elarge = [(u, v) for (u, v, d) in G.edges(data=True) if d["weight"] > 0.5]
esmall = [(u, v) for (u, v, d) in G.edges(data=True) if d["weight"] <= 0.5]

pos = nx.spring_layout(G, seed=7)  # positions for all nodes - seed for reproducibility

# nodes
nx.draw_networkx_nodes(G, pos, node_size=700)

# edges
nx.draw_networkx_edges(G, pos, edgelist=elarge, width=6)
nx.draw_networkx_edges(
    G, pos, edgelist=esmall, width=6, alpha=0.5, edge_color="b", style="dashed"
)

# node labels
nx.draw_networkx_labels(G, pos, font_size=20, font_family="sans-serif")
# edge weight labels
edge_labels = nx.get_edge_attributes(G, "weight")
nx.draw_networkx_edge_labels(G, pos, edge_labels)

ax = plt.gca()
ax.margins(0.08)
plt.axis("off")
plt.tight_layout()
plt.show()

In [ ]:
# Find al possible paths between S and D and save it into paths list

paths = []

for i in nx.all_simple_paths(G, source='s', target='t'):
  print(i)
#paths

In [ ]:
# Create de,and and revenue data manually
revenue = {'item_1': 50, 'item_2': 20, 'item_3': 30, 'item_4': 40, 'item_5': 50} 
bw_dem = {'item_1': 1000, 'item_2': 2000, 'item_3': 3000, 'item_4': 4000, 'item_5': 5000} 
# capacities = {'path_1': 1000, 'path_2': 6000, 'path_3': 8000}. # Knapsack capacities

classes = ['item_1', 'item_2', 'item_3', 'item_4', 'item_5']
# paths = ['path_1', 'path_2', 'path_3']

In [ ]:
# Generate paths (knapsacks) between S and D

# Find min cut for every path

import random
from random import randrange
paths = []

def find_cut(path):                        # a function to get the edge with minimum capacity in each path
  min = 1000000                            # a random value to maintain minimum capacity record in each path
  i = 0                                    # index for keeping path number
  x = 0                                    # u node of an edge
  y = 1                                    # v node of an edge
                                           # print("Path id: ", i )
  for i in range(len(path) - 1):           # to get u and v nodes of every edge
                                           # print(path[x], path[y])
    d = G.get_edge_data(path[x], path[y])  # get edge record from dictionary
    val = d['weight']                      # get the capacity value of edge
    if val < min:                          # compare the capacity value of edge to see if it is minimum until now
      min = val                            # keep the minimum value
    x = x + 1
    y = y + 1
    i = i + 1
                                          # print("Min cut is", min, " for path : ", path)
  return min


def path_d():                             # a fucntion to keep dicitonary of paths (knapsacks)           
   z = 0                                  # index for maintaining path id
   mydic = {}
   path_id = []                           # list to only keep the path id
   for path in nx.all_simple_paths(G, source='s', target='t'):               # a built-in function of networkx for finding paths
     path_id.append('path_'+str(z))
                                          #print("Path id: ", path_id[z])
     # paths.append(path)                 # already defined above
     #print("Path nodes:", path)

     min_cut = find_cut(path)             # cal the function to get minimum capacity edge of every path
     #print("Path id: ", path_id[z], "Path: ", path , "Min cut: ", min_cut)
     # mydic['path_'+str(z)] = randrange(1000,5000) # was used to assign random min edge capacity to each path
     mydic['path_'+str(z)] = min_cut     # assign minimum cut value here to each path
     z = z + 1
   return mydic                          # the output is path ids with minimum edge capacities


def path_l():                            # function for keeping the path ids that will be used if defining contraint
   z = 0       
   mylist = []
   for i in nx.all_simple_paths(G, source='s', target='t'):
     mylist.append('path_'+str(z))
     z = z + 1
   return mylist                         # the output is path ids

path_dic = path_d()
path_list = path_l()
# print("Path dic: ", path_dic)
# print("Path list: ", path_list)

In [ ]:
# Our data are listed belows, ready for apply multi-knapsack optimization to find the optimal placements

print("Demands: ", bw_dem)
print("Revenues: ", revenue)
print("Classes: ", classes)

print("Path dic: ", path_dic)
print("Path list: ", path_list)

In [ ]:
# Apply optimization here

import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# This value above are just exemples
m = gp.Model('ks')

# Insert the decision variables
x = m.addVars(classes, path_list, vtype = gp.GRB.BINARY)

# Define the objective function
m.setObjective(gp.quicksum(x[i, j] * revenue[i] for i in classes for j in path_list), sense=gp.GRB.MAXIMIZE)

# Constraint 1: satisfy the demand once, assign demand to only one knapsack at most
c1 = m.addConstrs(gp.quicksum(x[i,j] for j in path_list) <= 1 for i in classes)

# Constraint 2: respect the path capacity
c2 = m.addConstrs(gp.quicksum(x[i,j] * bw_dem[i] for i in classes) <= path_dic[j] for j in path_list)

# Run the model
m.optimize()
m.setParam(GRB.Param.OutputFlag, 0)

# Status checking
status = m.Status
if status in (GRB.INF_OR_UNBD, GRB.INFEASIBLE, GRB.UNBOUNDED):
    print('The model cannot be solved because it is infeasible or '
            'unbounded')
    sys.exit(1)

if status != GRB.OPTIMAL:
    print('Optimization was stopped with status ' + str(status))
    sys.exit(1)

In [ ]:
# Display optimal values of decision variables
for v in m.getVars():
    if v.x > 1e-6:
        print(v.varName, v.x)

# Display optimal total matching score
print('Total matching score: ', m.objVal)

In [ ]:
x